# BTTH04: Ẩn dữ liệu trên ảnh bằng phương pháp LSB

Võ Phương Hòa - 1412192

---

## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file). Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Mục tiêu chính ở đây là học, học một cách chân thật; nếu bạn được 5 điểm nhưng bạn làm bài một cách chân thật thì vẫn tốt hơn nhiều so với 10 điểm mà không chân thật. Bạn có thể thảo luận với các bạn khác; nhưng bài làm phải là của chính bạn, dựa trên sự hiểu của chính bạn, lời văn là của chính bạn, code là của chính bạn. Bạn không nên đọc bài làm của các bạn năm trước. Nếu bị phát hiện gian lận thì bạn sẽ bị 0 điểm cho toàn bộ phần thực hành.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Kernel` - `Restart & Run All` để restart và chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Kernel` - `Restart & Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt: file `Ex04-DataHiding-Img-LSB.ipynb`, và các file dữ liệu mà mình đính kèm (`cover.bmp`, `msg.txt`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [1]:
%matplotlib inline
import numpy as np
import bitarray
from scipy.misc import imread, imsave
import matplotlib.pyplot as plt
# You can also import other things ...

## 3. Hàm nhúng
(Khi code, để ý kiểu dữ liệu của các mảng numpy)

In [2]:
def embed_lsb(cover_img_file, msg_file, num_lsbs, stego_img_file):
    """
    Embeds a message into a cover image using the LSB method.
    
    Parameters
    ----------
    cover_img_file : string
        The name of the cover img file.
    msg_file : string
        The name of the message file.
    num_lsbs : int
        The number of LSBs used to embed.
    stego_img_file : string
        The name of the stego img file.
        
    Returns (if it's possible to embed)
    -----------------------------------
    (mae, max_num_bits) : tuple
        mae : float
            Mean Absolute Error between stego img and cover img;
            mae = np.mean(np.abs(stego_img - cover_img)).
        max_num_bits : int
            The max number of bits we can embed; 
            max_num_bits = num_lsbs * cover_img.size().       
    """
    # TODO
    # 1. Read cover img file
    cover_img = imread(cover_img_file)
        
    # 2. Read msg file and process
    # 2.1. Read msg file
    f = open(msg_file, 'r')
    msg_text = f.read()
    f.close()
    
    # 2.2 Convert msg to msg bits
    msg_bits = bitarray.bitarray()
    msg_bits.frombytes(msg_text)
    
    # 2.3. Check if it is possible to embed
    len_msg_bits = len(msg_bits)
    max_num_bits = num_lsbs * cover_img.size
    if (len_msg_bits >= max_num_bits):
        return (0, 0)
    
    # 2.4. Add '100...' to msg bits
    msg_bits.extend('1' + '0' * (max_num_bits - len_msg_bits - 1))
   
    # 2.5. Convert msg bits to msg img
    msg_bits_str = msg_bits.to01()
    msg_img = np.array([int(msg_bits_str[i: i + num_lsbs], 2) for i in range(0, len(msg_bits), num_lsbs)])
    msg_img = msg_img.reshape(cover_img.shape)
    
    # 3. Embed msg img into cover img
    stego_img = (cover_img >> num_lsbs << num_lsbs) + msg_img
    
    # 4. Write stego img to file
    imsave(stego_img_file, np.asarray(stego_img).astype('uint8'))
    
    # 5. Init result
    mae = np.mean(np.abs(stego_img - cover_img))
    t = (mae, max_num_bits)

    return t

## 4. Hàm rút trích

### 4.1. Hàm chuyển đổi số thập phân thành số nhị phân

In [3]:
def dec2bin(decimal):
    """
    Convert decimal to a binary representation.
    
    Parameter
    ---------------
    decimal: int
        Decimal need to be converted to a binary representation
    
    Return
    ---------------
    binary_str: string
        Binary representation of decimal
    """
    binary_str = ''
    if (decimal == 0):
        binary_str = '0'
    else:
        while (decimal > 0):
            binary_str = str(decimal % 2) + binary_str
            decimal /= 2
    
    return binary_str   

### 4.2. Hàm rút trích

In [4]:
def extract_lsb(stego_img_file, num_lsbs, extracted_msg_file):
    """
    Extracts the message from a stego image using the LSB method.
    
    Parameters
    ----------
    stego_img_file : string
        The name of the stego img file.
    num_lsbs : int
        The number of LSBs used to embed.
    extracted_msg_file : string
        The name of the extracted message file.
    """
    # TODO
    # 1. Read stego img file
    stego_img = imread(stego_img_file)
        
    # 2. Extract msg bits str from stego img
    # 2.1. Take msg img from stego img and num_lsbs
    msg_img = stego_img - (stego_img >> num_lsbs << num_lsbs)
   
    # 2.2. Convert msg img to msg bits
    msg_img = msg_img.reshape(msg_img.size)
    len_msg_img = len(msg_img)
    msg_bits_str = ''
    for i in range(0, len_msg_img):
        msg_bits = dec2bin(msg_img[i])
        msg_bits_str += '0' * (num_lsbs - len(msg_bits)) + msg_bits
    
    # 2.3. Find index of the last bit 1 of msg bits str,
    # create a new string with content similar to the old string
    # but eliminate the last bits '1000 ...' based on the index of the last bit 1
    index = len(msg_bits_str) - 1     
    while index >= 0:
        if msg_bits_str[index] == '1':
            break
        index -= 1
    msg_bits_str = msg_bits_str[0:index]
    
    # 3. Convert msg bits to msg text
    msg_bits = bitarray.bitarray(msg_bits_str)
    msg_text = msg_bits.tobytes()
    
    # 4. Write msg text to file
    f = open(extracted_msg_file, 'w')
    f.write(msg_text)
    f.close()
    
    return

## 5. Chạy thí nghiệm

Đầu tiên, bạn sẽ gọi hàm nhúng và hàm rút trích ở trên với file cover `cover.bmp` và file message `msg.txt` mà mình đính kèm; bạn sử dụng `num_lsbs = 1` và để file stego có đuôi là `*.bmp` (để tránh bị mất mát thông tin khi ghi ảnh xuống file). Để minh chứng hàm nhúng và hàm rút trích đã được cài đặt đúng, bạn cũng cần mở file message ban đầu và file message được rút trích, và in nội dung của hai file này ra.

In [5]:
# TODO
input_1 = 'cover.bmp'
input_2 = 'msg.txt'
output_1 = 'stego.bmp'
num_lsbs = 1

t = embed_lsb(input_1, input_2, num_lsbs, output_1)
if (t != (0, 0)):
    output_2 = 'extracted.txt'
    extract_lsb(output_1, num_lsbs, output_2)
    
    f = open(input_2, 'r')
    msg_text = f.read()
    f.close()
    
    f = open(output_2, 'r')
    extracted_text = f.read()
    f.close()
    
    print msg_text
    print '---------------------------------'
    print extracted_text

OVER YOU
[Watch it here: 
https://www.youtube.com/watch?v=y24DLOdFpt0 (by Cassadee Pope)
https://www.youtube.com/watch?v=cGABJorRJ50 (by Blake Shelton)]

Weather man said it iss gonna snow
By now I should be used to the cold
Mid-February shouldnt be so scary
It was only December
I still remember the presents, the tree, you and me

But you went away
How dare you?
I miss you
They say I will be OK
But Im not going to ever get over you

Living alone here in this place
I think of you, and Im not afraid
Your favorite records make me feel better
Cause you sing along
With every song
I know you didnt mean to give them to me

But you went away
How dare you?
I miss you
They say Ill be OK
But Im not going to ever get over you
---------------------------------
OVER YOU
[Watch it here: 
https://www.youtube.com/watch?v=y24DLOdFpt0 (by Cassadee Pope)
https://www.youtube.com/watch?v=cGABJorRJ50 (by Blake Shelton)]

Weather man said it iss gonna snow
By now I should be used to the cold
Mid-February shou

Kế đến, bạn sẽ viết một đoạn code để thử nghiệm hàm nhúng với `num_lsbs = 1, 2, 3, 4, 5, 6, 7, 8`. In ra màn hình `mae` và `max_num_bits` ứng với từng giá trị của `num_lsbs`; để trực quan, bạn có thể vẽ thêm đồ thị (bằng code). Cho nhận xét về kết quả.

In [6]:
# TODO
input_1 = 'cover.bmp'
input_2 = 'msg.txt'
output_1 = 'stego.bmp'

for num_lsbs in range(1, 9):
    t = embed_lsb(input_1, input_2, num_lsbs, output_1)
    if (t != (0, 0)):
        output_2 = 'extracted.txt'
        extract_lsb(output_1, num_lsbs, output_2)
    
    print 'num_lsbs = ', num_lsbs, ':', t

num_lsbs =  1 : (0.50116348266601563, 262144)
num_lsbs =  2 : (1.4982681274414062, 524288)
num_lsbs =  3 : (3.4891510009765625, 786432)
num_lsbs =  4 : (7.4722480773925781, 1048576)
num_lsbs =  5 : (15.517425537109375, 1310720)
num_lsbs =  6 : (31.714927673339844, 1572864)
num_lsbs =  7 : (58.623805999755859, 1835008)
num_lsbs =  8 : (123.81041717529297, 2097152)


TODO:
### Nhận xét:
Theo chiều tăng num_lsbs, các giá trị mae và max_num_bits tăng dần.

Trong đó:
- Độ lỗi giữa cover_img và stego_img **mae** $\approx \frac{\displaystyle \sum_{i=0}^n(n)}{n + 1}$, với n = $2^a -1$ (a = num_lsbs)

    Độ lỗi càng lớn thì ảnh sau ẩn dữ liệu (stego_img) càng khác so với ảnh gốc (cover_img), khi đó việc ẩn dữ liệu không còn hiệu quả như mong muốn.
- **max_num_bits = img.size x num_lsbs** (ở đây, kích thước ảnh 'img' thường là 512 x 512)
    
    Số bit ẩn trên mỗi byte càng lớn thì dữ liệu có thể ẩn được càng lớn. Nhưng sau ẩn dữ liệu, byte dữ liệu càng khác so với byte gốc, từ đó ảnh sau ẩn dữ liệu sẽ ngày càng khác so với ảnh gốc.

Tóm lại, khi num_lsbs càng lớn, ta càng giấu được nhiều dữ liệu hơn vào ảnh. Tuy nhiên, độ lỗi giữ ảnh gốc và ảnh sau ẩn dữ liệu ngày càng cao. Điều đó cũng đồng nghĩa ảnh sau khi ẩn dữ liệu sẽ ngày càng khác ảnh gốc và hiệu quả ẩn dữ liệu giảm xuống.



---

Enjoy coding :-)